### Load VALID_JOIN, 
#### the table produced by joining VALID_LOG_DATA and VALID_KEYSTROKES

In [2]:
from configparser import ConfigParser
from sqlalchemy import create_engine
import pandas as pd
from geolite2 import geolite2
import geopandas as gpd
import pycountry
import re
import seaborn as sns
import matplotlib.pyplot as plt
get_ipython().magic('matplotlib inline')
sns.set(color_codes=True)
%load_ext autotime

def read_db_config(filename='config-dump.ini', section='mysql'):
    """ Read database configuration file and return a dictionary object
    :param filename: name of the configuration file
    :param section: section of database configuration
    :return: a dictionary of database parameters
    """
    # create parser and read ini configuration file
    parser = ConfigParser()
    parser.read(filename)
 
    # get section, default to mysql
    db = {}
    if parser.has_section(section):
        items = parser.items(section)
        for item in items:
            db[item[0]] = item[1]
    else:
        raise Exception('{0} not found in the {1} file'.format(section, filename))
 
    return db

dbconfig = read_db_config()
CONN = 'mysql+mysqlconnector://'+dbconfig['user']+':'+dbconfig['pwd']+'@'+dbconfig['host']+':'+dbconfig['port'] + '/'+dbconfig['dbname']
engine = create_engine(CONN)

time: 3.59 s


In [4]:
DATA_QUERY = "SELECT * FROM VALID_JOIN" 
join_data = pd.read_sql_query(DATA_QUERY, engine)
join_data.fillna(0, inplace=True)
# time: 32min 31s

time: 32min 31s


In [5]:
join_data.tail()

TIMESTAMP    PRESS_TIME  RELEASE_TIME     TYPE KEY    CODE DATA  \
22829451  1539673295047  0.000000e+00  0.000000e+00    input  -1      -1    t   
22829452  1539673295169  0.000000e+00  0.000000e+00    keyup   t    KeyT   -1   
22829453  1539673295430  1.539673e+12  1.539673e+12  keydown   .  Period   -1   
22829454  1539673295442  0.000000e+00  0.000000e+00    input  -1      -1    .   
22829455  1539673295542  0.000000e+00  0.000000e+00    keyup   .  Period   -1   

         LETTER  KEYCODE  TEST_SECTION_ID SCREEN_ORIENTATION  LOG_DATA_ID  \
22829451      0      0.0           386173                  0     51537494   
22829452      0      0.0           386173                  0     51537495   
22829453      .    190.0           386173                  0     51537496   
22829454      0      0.0           386173                  0     51537497   
22829455      0      0.0           386173                  0     51537498   

          KEYSTROKE_ID  
22829451           0.0  
22829452           0.0  
22829453    16793169.0  
22829454           0.0  
22829455           0.0

time: 137 ms


## Get bigrams and IKI intervals from logged sample

#### Observations (from 1% SAMPLE): 
#### From captured log_data keydown events, ~30% were not captured (Unidentified KEY, empty KEYCODE).
For example, see 
TEST_SECTION_ID 10102 from PARTICIPANT_ID 3058 (Samsung mobile)<br>
Check: <br>
psksld[psksld.TYPE == 'keyup'].CODE.value_counts()<br>
psksld[psksld.TEST_SECTION_ID == 10102]<br>



In [7]:
# ks.KEYCODE.value_counts(), ks.LETTER.value_counts() : most of the keycodes and letters are undefined
kd_CODE_counts = join_data[join_data.TYPE == 'keydown'].CODE.value_counts()

time: 40.3 s


In [10]:
join_data.to_pickle("join_data.pkl")

time: 4min 13s


In [8]:
kd_CODE_counts

                  3872176
Space              564999
KeyE               323710
KeyT               234128
Backspace          213452
KeyA               209408
KeyO               207449
KeyI               199981
-1                 176418
KeyN               174819
KeyS               165451
KeyR               144623
KeyH               131194
KeyL               113932
KeyD                93635
KeyU                77632
KeyW                69655
KeyM                68435
KeyC                66573
KeyY                61981
KeyG                54800
KeyF                53259
Period              50436
KeyP                48948
KeyB                41952
KeyK                35640
KeyV                30113
Lang1               20326
KeyJ                10089
Quote                9810
                   ...   
ArrowDown              13
Numpad1                13
Numpad2                10
NumpadEnter            10
AltRight                9
NumLock                 9
F5                      9
F12         

time: 83.7 ms


In [11]:
### IKI 
# Load VALID_KEYSTROKES and 


TIMESTAMP    PRESS_TIME  RELEASE_TIME     TYPE KEY    CODE DATA  \
22829426  1539673291960  1.539673e+12  1.539673e+12  keydown   o    KeyO   -1   
22829427  1539673291966  0.000000e+00  0.000000e+00    input  -1      -1    o   
22829428  1539673292116  0.000000e+00  0.000000e+00    keyup   o    KeyO   -1   
22829429  1539673292243  1.539673e+12  1.539673e+12  keydown   r    KeyR   -1   
22829430  1539673292250  0.000000e+00  0.000000e+00    input  -1      -1    r   
22829431  1539673292346  0.000000e+00  0.000000e+00    keyup   r    KeyR   -1   
22829432  1539673292872  1.539673e+12  1.539673e+12  keydown   d    KeyD   -1   
22829433  1539673292877  0.000000e+00  0.000000e+00    input  -1      -1    d   
22829434  1539673292970  0.000000e+00  0.000000e+00    keyup   d    KeyD   -1   
22829435  1539673293111  1.539673e+12  1.539673e+12  keydown   e    KeyE   -1   
22829436  1539673293114  0.000000e+00  0.000000e+00    input  -1      -1    e   
22829437  1539673293207  0.000000e+00  0.000000e+00    keyup   e    KeyE   -1   
22829438  1539673293537  1.539673e+12  1.539673e+12  keydown   r    KeyR   -1   
22829439  1539673293548  0.000000e+00  0.000000e+00    input  -1      -1    r   
22829440  1539673293633  0.000000e+00  0.000000e+00    keyup   r    KeyR   -1   
22829441  1539673293726  1.539673e+12  1.539673e+12  keydown       Space   -1   
22829442  1539673293732  0.000000e+00  0.000000e+00    input  -1      -1        
22829443  1539673293843  0.000000e+00  0.000000e+00    keyup       Space   -1   
22829444  1539673294275  1.539673e+12  1.539673e+12  keydown   y    KeyY   -1   
22829445  1539673294281  0.000000e+00  0.000000e+00    input  -1      -1    y   
22829446  1539673294359  0.000000e+00  0.000000e+00    keyup   y    KeyY   -1   
22829447  1539673294584  1.539673e+12  1.539673e+12  keydown   e    KeyE   -1   
22829448  1539673294591  0.000000e+00  0.000000e+00    input  -1      -1    e   
22829449  1539673294700  0.000000e+00  0.000000e+00    keyup   e    KeyE   -1   
22829450  1539673295040  1.539673e+12  1.539673e+12  keydown   t    KeyT   -1   
22829451  1539673295047  0.000000e+00  0.000000e+00    input  -1      -1    t   
22829452  1539673295169  0.000000e+00  0.000000e+00    keyup   t    KeyT   -1   
22829453  1539673295430  1.539673e+12  1.539673e+12  keydown   .  Period   -1   
22829454  1539673295442  0.000000e+00  0.000000e+00    input  -1      -1    .   
22829455  1539673295542  0.000000e+00  0.000000e+00    keyup   .  Period   -1   

         LETTER  KEYCODE  TEST_SECTION_ID SCREEN_ORIENTATION  LOG_DATA_ID  \
22829426      o     79.0           386173                  0     51537469   
22829427      0      0.0           386173                  0     51537470   
22829428      0      0.0           386173                  0     51537471   
22829429      r     82.0           386173                  0     51537472   
22829430      0      0.0           386173                  0     51537473   
22829431      0      0.0           386173                  0     51537474   
22829432      d     68.0           386173                  0     51537475   
22829433      0      0.0           386173                  0     51537476   
22829434      0      0.0           386173                  0     51537477   
22829435      e     69.0           386173                  0     51537478   
22829436      0      0.0           386173                  0     51537479   
22829437      0      0.0           386173                  0     51537480   
22829438      r     82.0           386173                  0     51537481   
22829439      0      0.0           386173                  0     51537482   
22829440      0      0.0           386173                  0     51537483   
22829441            32.0           386173                  0     51537484   
22829442      0      0.0           386173                  0     51537485   
22829443      0      0.0           386173                  0     51537486   
22829444      y     89.0           3861

time: 128 ms
